In [ ]:
pip install pomegranate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pomegranate-0.14.8.tar.gz (4.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pomegranate: filename=pomegranate-0.14.8-cp38-cp38-linux_x86_64.whl size=22140240 sha256=15a652be3d98aef59edba8dd95282b26b31903c1b2c85a972ac52f4adfca759c
  Stored in directory: /root/.cache/pip/wheels/e5/39/08/5dac86e6d7ef4c6b9f337b0913f1bf96a8b32ff45f833b067f
Successfully built pomegranate


# Estructura de las busquedas

In [ ]:
class Nodo():
    def __init__(self, estado, padre, accion):
        self.estado = estado
        self.padre = padre
        self.accion = accion

class Frontera():
    def __init__(self):
        self.frontera =[]
    
    def empty(self):
        return (len(self.frontera) == 0)

    def add(self, nodo):
        self.frontera.append(nodo)
    
    def eliminar(self):
        # LIFO o FIFO
        pass

    def contiene_estado(self, estado):
        return any(nodo.estado == estado for nodo in self.frontera)

class Pila(Frontera):
    def eliminar(self):
        # Termina la busqueda si la frontera esta vacia
        if self.empty():        
            raise Exception("Frontera vacia")
        else:
            # Guardamos el ultimo item en la lista 
            # (el cual es el nodo recientemente añadido)
            nodo = self.frontera[-1]
            # Guardamos todos los items excepto el 
            # ultimo (eliminamos)
            self.frontera = self.frontera[:-1]
            return nodo
    
class Cola(Frontera):
    def eliminar(self):
        # Termina la busqueda si la frontera esta vacia
        if self.empty():        
            raise Exception("Frontera vacia")
        else:
            # Guardamos el primer item en la lista 
            # (el cual es el nodo añadido de primero)
            nodo = self.frontera[0]
            # Guardamos todos los items excepto el 
            # primero (eliminamos)
            self.frontera = self.frontera[1:]
            return nodo

# SimpleProblemSolvingAgentProgram

In [ ]:
# from search import *

class SimpleProblemSolvingAgentProgram:
    """
    [Figure 3.1]
    Abstract framework for a problem-solving agent.
    """

    def __init__(self, initial_state=None):
        """State is an abstract representation of the state
        of the world, and seq is the list of actions required
        to get to a particular state from the initial state(root)."""
        self.state = initial_state
        self.seq = []

    def __call__(self, percept):
        """[Figure 3.1] Formulate a goal and problem, then
        search for a sequence of actions to solve it."""
        self.state = self.update_state(self.state, percept)
        if not self.seq:
            goal = self.formulate_goal(self.state)
            problem = self.formulate_problem(self.state, goal)
            self.seq = self.search(problem)
            if not self.seq:
                return None
        return self.seq
        # return self.seq.pop(0)

    def update_state(self, state, percept):
        raise NotImplementedError

    def formulate_goal(self, state):
        raise NotImplementedError

    def formulate_problem(self, state, goal):
        raise NotImplementedError

    def search(self, problem):
        raise NotImplementedError

# Modulo logic

In [ ]:
import itertools


class Sentence():

    def evaluate(self, model):
        """Evaluates the logical sentence."""
        raise Exception("nothing to evaluate")

    def formula(self):
        """Returns string formula representing logical sentence."""
        return ""

    def symbols(self):
        """Returns a set of all symbols in the logical sentence."""
        return set()

    @classmethod
    def validate(cls, sentence):
        if not isinstance(sentence, Sentence):
            raise TypeError("must be a logical sentence")

    @classmethod
    def parenthesize(cls, s):
        """Parenthesizes an expression if not already parenthesized."""
        def balanced(s):
            """Checks if a string has balanced parentheses."""
            count = 0
            for c in s:
                if c == "(":
                    count += 1
                elif c == ")":
                    if count <= 0:
                        return False
                    count -= 1
            return count == 0
        if not len(s) or s.isalpha() or (
            s[0] == "(" and s[-1] == ")" and balanced(s[1:-1])
        ):
            return s
        else:
            return f"({s})"


class Symbol(Sentence):

    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return isinstance(other, Symbol) and self.name == other.name

    def __hash__(self):
        return hash(("symbol", self.name))

    def __repr__(self):
        return self.name

    def evaluate(self, model):
        try:
            return bool(model[self.name])
        except KeyError:
            raise EvaluationException(f"variable {self.name} not in model")

    def formula(self):
        return self.name

    def symbols(self):
        return {self.name}


class Not(Sentence):
    def __init__(self, operand):
        Sentence.validate(operand)
        self.operand = operand

    def __eq__(self, other):
        return isinstance(other, Not) and self.operand == other.operand

    def __hash__(self):
        return hash(("not", hash(self.operand)))

    def __repr__(self):
        return f"Not({self.operand})"

    def evaluate(self, model):
        return not self.operand.evaluate(model)

    def formula(self):
        return "¬" + Sentence.parenthesize(self.operand.formula())

    def symbols(self):
        return self.operand.symbols()


class And(Sentence):
    def __init__(self, *conjuncts):
        for conjunct in conjuncts:
            Sentence.validate(conjunct)
        self.conjuncts = list(conjuncts)

    def __eq__(self, other):
        return isinstance(other, And) and self.conjuncts == other.conjuncts

    def __hash__(self):
        return hash(
            ("and", tuple(hash(conjunct) for conjunct in self.conjuncts))
        )

    def __repr__(self):
        conjunctions = ", ".join(
            [str(conjunct) for conjunct in self.conjuncts]
        )
        return f"And({conjunctions})"

    def add(self, conjunct):
        Sentence.validate(conjunct)
        self.conjuncts.append(conjunct)

    def evaluate(self, model):
        return all(conjunct.evaluate(model) for conjunct in self.conjuncts)

    def formula(self):
        if len(self.conjuncts) == 1:
            return self.conjuncts[0].formula()
        return " ∧ ".join([Sentence.parenthesize(conjunct.formula())
                           for conjunct in self.conjuncts])

    def symbols(self):
        return set.union(*[conjunct.symbols() for conjunct in self.conjuncts])


class Or(Sentence):
    def __init__(self, *disjuncts):
        for disjunct in disjuncts:
            Sentence.validate(disjunct)
        self.disjuncts = list(disjuncts)

    def __eq__(self, other):
        return isinstance(other, Or) and self.disjuncts == other.disjuncts

    def __hash__(self):
        return hash(
            ("or", tuple(hash(disjunct) for disjunct in self.disjuncts))
        )

    def __repr__(self):
        disjuncts = ", ".join([str(disjunct) for disjunct in self.disjuncts])
        return f"Or({disjuncts})"

    def evaluate(self, model):
        return any(disjunct.evaluate(model) for disjunct in self.disjuncts)

    def formula(self):
        if len(self.disjuncts) == 1:
            return self.disjuncts[0].formula()
        return " ∨  ".join([Sentence.parenthesize(disjunct.formula())
                            for disjunct in self.disjuncts])

    def symbols(self):
        return set.union(*[disjunct.symbols() for disjunct in self.disjuncts])


class Implication(Sentence):
    def __init__(self, antecedent, consequent):
        Sentence.validate(antecedent)
        Sentence.validate(consequent)
        self.antecedent = antecedent
        self.consequent = consequent

    def __eq__(self, other):
        return (isinstance(other, Implication)
                and self.antecedent == other.antecedent
                and self.consequent == other.consequent)

    def __hash__(self):
        return hash(("implies", hash(self.antecedent), hash(self.consequent)))

    def __repr__(self):
        return f"Implication({self.antecedent}, {self.consequent})"

    def evaluate(self, model):
        return ((not self.antecedent.evaluate(model))
                or self.consequent.evaluate(model))

    def formula(self):
        antecedent = Sentence.parenthesize(self.antecedent.formula())
        consequent = Sentence.parenthesize(self.consequent.formula())
        return f"{antecedent} => {consequent}"

    def symbols(self):
        return set.union(self.antecedent.symbols(), self.consequent.symbols())


class Biconditional(Sentence):
    def __init__(self, left, right):
        Sentence.validate(left)
        Sentence.validate(right)
        self.left = left
        self.right = right

    def __eq__(self, other):
        return (isinstance(other, Biconditional)
                and self.left == other.left
                and self.right == other.right)

    def __hash__(self):
        return hash(("biconditional", hash(self.left), hash(self.right)))

    def __repr__(self):
        return f"Biconditional({self.left}, {self.right})"

    def evaluate(self, model):
        return ((self.left.evaluate(model)
                 and self.right.evaluate(model))
                or (not self.left.evaluate(model)
                    and not self.right.evaluate(model)))

    def formula(self):
        left = Sentence.parenthesize(str(self.left))
        right = Sentence.parenthesize(str(self.right))
        return f"{left} <=> {right}"

    def symbols(self):
        return set.union(self.left.symbols(), self.right.symbols())


def model_check(knowledge, query):
    """Checks if knowledge base entails query."""

    def check_all(knowledge, query, symbols, model):
        """Checks if knowledge base entails query, given a particular model."""

        # If model has an assignment for each symbol
        if not symbols:

            # If knowledge base is true in model, then query must also be true
            if knowledge.evaluate(model):
                return query.evaluate(model)
            return True
        else:

            # Choose one of the remaining unused symbols
            remaining = symbols.copy()
            p = remaining.pop()

            # Create a model where the symbol is true
            model_true = model.copy()
            model_true[p] = True

            # Create a model where the symbol is false
            model_false = model.copy()
            model_false[p] = False

            # Ensure entailment holds in both models
            return (check_all(knowledge, query, remaining, model_true) and
                    check_all(knowledge, query, remaining, model_false))

    # Get all symbols in both knowledge and query
    symbols = set.union(knowledge.symbols(), query.symbols())

    # Check that knowledge entails query
    return check_all(knowledge, query, symbols, dict())

# lógica del juego

In [ ]:
import random, time, termcolor

Elon = Symbol('Elon')
Greta = Symbol('Greta')
Jeff = Symbol('Jeff')
Podrick = Symbol('Podrick')
Teresa = Symbol('Teresa')
Alfred = Symbol('Alfred')
Bruno = Symbol('Bruno')
personajes = [Elon, Greta, Jeff, Podrick, Teresa, Alfred]
sospechosos = personajes.copy()
if Elon in personajes:
  print(True)
# Habitaciones de la mansión
comedor = Symbol('Comedor')
hab_jeff = Symbol('Habitación de Jeff')
hab_bruno = Symbol('Habitacion de Bruno')
hab_huesped = Symbol('Habitacion de huéspedes')
sala = Symbol('Sala de estár')
sotano = Symbol('Sotano')
balcon = Symbol('Balcón')
patio = Symbol('Patio')
cocina = Symbol('Cocina')
habitaciones = [comedor, hab_jeff, hab_bruno, hab_huesped, sala, sotano, balcon, patio, cocina]

# Evidencias
revolver = Symbol("Revolver")
hacha = Symbol("Hacha")
veneno = Symbol("Veneno")
cuchillo = Symbol("Cuchillo")
armas = [revolver, hacha, veneno, cuchillo]

# Símbolos
simbolos = personajes + habitaciones + armas

parametros = [habitaciones,personajes,armas]
evidencias = []

#Generar evidencias al azar

for i in range(len(habitaciones)):
  n_evidencias = random.randint(0,2)
  evidencia = []
  #Seleccionamos una habitacion al azar
  habitacion = random.choice(parametros[0])
  ele = parametros[0].remove(habitacion)
  evidencia.append(habitacion)
  for j in range(n_evidencias):
    #Seleccionamos un arma o un personaja o la dos

    if not len(parametros[1]) == 0 or not len(parametros[2]) == 0:
      number = j+1
    if len(parametros[1]) == 0:
      number = 2
    if len(parametros[2]) == 0:
      number = 1
    if len(parametros[1]) == 0 and len(parametros[2]) == 0:
      break
      
    eleccion = random.choice(parametros[number])
    # Remover eleccion
    x = parametros[number].remove(eleccion)
    evidencia.append(eleccion)

  #Añadimos evidencia  
  evidencias.append(evidencia)


habitaciones = [[["No visitado",evidencias[0]], ["No visitado", evidencias[1]], ["No visitado", evidencias[2]]], 
                [["No visitado", evidencias[3]], ["No visitado",evidencias[4]], ["No visitado",evidencias[5]]], 
                [["No visitado",evidencias[6]],["No visitado",evidencias[7]], ["No visitado", evidencias[8]]] ]


objetivo =     [[["Check",evidencias[0]], ["Check", evidencias[1]], ["Check", evidencias[2]]], 
                [["Check", evidencias[3]], ["Check",evidencias[4]], ["Check",evidencias[5]]], 
                [["Check",evidencias[6]],["Check",evidencias[7]], ["Check", evidencias[8]]] ]

state1 = [(0, 0),habitaciones]


# Parametros del juego

spected_killer = And(
    Or(Elon, Greta, Jeff, Podrick, Teresa, Alfred))
#     Or(hacha, revolver, veneno, cuchillo),
#     Or(hab_bruno, hab_jeff, hab_huesped, balcon, cocina, sotano, patio, sala, comedor)
# )

# Restricciones para que haya un solo asesino
spected_killer.add(Or(
    And(Elon, Not(Greta), Not(Jeff), Not(Podrick), Not(Teresa), Not(Alfred)),
    And(Greta, Not(Elon), Not(Jeff), Not(Podrick), Not(Teresa), Not(Alfred)),
    And(Jeff, Not(Greta), Not(Elon), Not(Podrick), Not(Teresa), Not(Alfred)),
    And(Podrick, Not(Jeff), Not(Greta), Not(Elon), Not(Teresa), Not(Alfred)),
    And(Teresa, Not(Podrick), Not(Jeff), Not(Greta), Not(Elon), Not(Alfred)), 
    And(Alfred, Not(Teresa), Not(Podrick), Not(Jeff), Not(Greta), Not(Elon)),
))

# Restricciones para que haya solo una habitación
'''spected_killer.add(Or(
    And(hab_bruno, Not(hab_jeff), Not(hab_huesped), Not(patio), Not(sotano), Not(cocina), Not(comedor), Not(sala), Not(balcon)),
    And(hab_jeff, Not(hab_bruno), Not(hab_huesped), Not(patio), Not(sotano), Not(cocina), Not(comedor), Not(sala), Not(balcon)),
    And(hab_huesped, Not(hab_jeff), Not(hab_bruno), Not(patio), Not(sotano), Not(cocina), Not(comedor), Not(sala), Not(balcon)),
    And(patio, Not(hab_jeff), Not(hab_bruno), Not(hab_huesped), Not(sotano), Not(cocina), Not(comedor), Not(sala), Not(balcon)),
    And(sotano, Not(hab_jeff), Not(hab_bruno), Not(hab_huesped), Not(patio), Not(cocina), Not(comedor), Not(sala), Not(balcon)),
    And(cocina, Not(hab_jeff), Not(hab_bruno), Not(hab_huesped), Not(patio), Not(sotano), Not(comedor), Not(sala), Not(balcon)),
    And(comedor, Not(hab_jeff), Not(hab_bruno), Not(hab_huesped), Not(patio), Not(sotano), Not(cocina), Not(sala), Not(balcon)),
    And(sala, Not(hab_jeff), Not(hab_bruno), Not(hab_huesped), Not(patio), Not(sotano), Not(cocina), Not(comedor), Not(balcon)),
    And(balcon, Not(hab_jeff), Not(hab_bruno), Not(hab_huesped), Not(patio), Not(sotano), Not(cocina), Not(comedor), Not(sala)),
))

# Restricciones para que haya solo un arma
spected_killer.add(Or(
    And(revolver, Not(hacha), Not(veneno), Not(cuchillo)),
    And(hacha, Not(revolver),  Not(hacha), Not(veneno)),
    And(veneno, Not(revolver),  Not(hacha), Not(cuchillo)),
    And(cuchillo, Not(revolver),  Not(hacha), Not(veneno))
))
'''

# Se descarta a Bruno ya que es la victima
spected_killer.add(
    Not(Bruno)
)

evidencias

True


[[Habitacion de Bruno, Elon],
 [Balcón],
 [Comedor],
 [Cocina],
 [Habitación de Jeff, Teresa],
 [Sala de estár, Greta, Hacha],
 [Patio, Jeff, Cuchillo],
 [Sotano],
 [Habitacion de huéspedes]]

# Agente

In [ ]:
class vacuumAgent(SimpleProblemSolvingAgentProgram):
        def __init__(self):
          self.check_knowledge(conocimiento)
          
        def update_state(self, state, percept):
            return percept

        def formulate_goal(self, state):
            goal = len(habitaciones)*len(habitaciones[0])
            return goal  

        def formulate_problem(self, state, goal):
            problem = state, goal
            return problem

        def check_knowledge(self,conocimiento):
            for symbol in simbolos:
                if model_check(conocimiento, symbol):
                    termcolor.cprint(f"\n{symbol}: SI", "green")
                    if (symbol in personajes):
                      flag = True                  
                        
                    # if (symbol in habitaciones):
                    #     print('Habitaciones')
                    # if (symbol in evidencias):
                    #     print('Evidencias')
                elif not model_check(conocimiento, Not(symbol)):
                    print(f"{symbol}: TAL VEZ") 

        def Logic(self,evidencias,conocimiento):

          print()
          print("Revisando")
          time.sleep(1)

          if len(evidencias)-1 == 2:
            if evidencias[1] in personajes and evidencias[2] in armas:
              print("Yes")
              spected_killer.add(Or(And(evidencias[0],evidencias[1],evidencias[2])))
            else:
              print(True)
              spected_killer.add(And(Not(evidencias[0]),Not(evidencias[1]),Not(evidencias[2])))

          elif len(evidencias)-1 == 1:
            spected_killer.add(And(Not(evidencias[0]),Not(evidencias[1])))
          else:
            spected_killer.add(Not(evidencias[0]))

          self.check_knowledge(conocimiento)
          
          
          
        
        def search(self, problem):

            self.num_explorados = 0
            state, goal = problem
            # Inicializamos la frontera para empezar en la posicion inicial
            start = Nodo(estado=state, padre=None, accion=None)
            frontera = Pila()
            frontera.add(start)
        
            # Inicializamos en conjunto explorado vacio
            self.explorado = set()
            self.cont = 0
            # Manetenmos el bucle hasta que encontremos la solución
            while True:

                # Si nada queda en la frontera, entonces no hya mas camino

                if frontera.empty():
                    print(self.cont)
                    raise Exception("No hay Solución")
            
                # Escogemos un nodo de la frontera
                nodo = frontera.eliminar()
                n,k = nodo.estado[0]
                self.num_explorados +=1
                if nodo.accion == "revisando":
                  n,k = nodo.estado[0]
                  self.cont += 1
                  nodo.estado[1][n][k][0] = "Check"
                  #Leemos evidencia

                  self.Logic(nodo.estado[1][n][k][1],spected_killer)


                # Si el nodo es el objetivo, entonces tenemos una solución
                if nodo.estado[1] == objetivo:
                    acciones = []
                    cel = []
                    # Rastreamos los nodos padre hasta la solución (objetivo hasta estado inicial)
                    while nodo.padre is not None:
                        acciones.append(nodo.accion)
                        cel.append(nodo.estado)
                        nodo = nodo.padre
                    acciones.reverse()
                    cel.reverse()
                    self.solucion = (acciones, cel)
                    return acciones
            
                # Marcamos el nodo como exploado
                n,k = nodo.estado[0]
                if not nodo.estado[1][n][k][0] == "No visitado":
                  self.explorado.add(nodo.estado[0])
                # Agregamos vecinos a la frontera
                for accion, estado in self.acciones_valida(nodo.estado):
                    if not frontera.contiene_estado(estado[0]) and estado[0] not in self.explorado:
                        hijo = Nodo(estado = estado, padre = nodo, accion=accion)
                        frontera.add(hijo)


        def acciones_valida(self, state):
          fila, col = state[0]
          self.altura = len(habitaciones)
          self.ancho = len(habitaciones[0])
          candidatos = [
              ("up", (fila -1, col)),
              ("down", (fila +1, col)),
              ("left", (fila, col -1)),
              ("right", (fila, col +1)),
              ("revisando", (fila, col))
              
          ]

          resultados = []
          for accion, (f,c) in candidatos:
              if 0<=f < self.altura and 0<=c < self.ancho:
                if state[1][f][c][0] == "No visitado":
                  resultados.append((accion,[(f,c), state[1]]))
                if not state[1][f][c][0] == "No visitado" and not accion == "revisando":
                  resultados.append((accion,[(f,c), state[1]]))
        
    
          return resultados

a = vacuumAgent(state1)

print(a(state1))

Jeff: TAL VEZ
Alfred: TAL VEZ
Comedor: TAL VEZ
Habitacion de Bruno: TAL VEZ
Habitacion de huéspedes: TAL VEZ
Patio: TAL VEZ
Cocina: TAL VEZ
Revolver: TAL VEZ
Cuchillo: TAL VEZ

Revisando
Jeff: TAL VEZ
Alfred: TAL VEZ
Comedor: TAL VEZ
Habitacion de huéspedes: TAL VEZ
Patio: TAL VEZ
Cocina: TAL VEZ
Revolver: TAL VEZ
Cuchillo: TAL VEZ
True

Revisando

Alfred: SI
Comedor: TAL VEZ
Habitacion de huéspedes: TAL VEZ
Cocina: TAL VEZ
Revolver: TAL VEZ

Revisando

Alfred: SI
Comedor: TAL VEZ
Habitacion de huéspedes: TAL VEZ
Revolver: TAL VEZ

Revisando
['right', 'right', 'down', 'left', 'left', 'revisando', 'down', 'revisando', 'right', 'revisando', 'right', 'revisando']


# Red bayesiana

In [ ]:
from pomegranate import *


Antecedentes = Node(DiscreteDistribution({
    "Ratero": 0.06,
    "Asesino": 0.001,
    "Neutro": 0.939
}), name="Antecedentes")


# Relación con el muerto
Relacion = Node(DiscreteDistribution({
    "Madre": 0.08,	
    "Padre": 0.08,	 
    "Hijo": 0.1,	
    "Hermano": 0.08,	
    "Amigo": 0.16,	
    "Abuelo": 0.08,	
    "Nieto": 0.14,	
    "Pareja": 0.04,	
    "Ninguna": 0.24
}), name="Relacion")


Consumo = Node(DiscreteDistribution({
    "Alcohol": 0.04,	
    "Droga": 0.01 ,
    "Normal": 0.95
}), name="Consumo")


# Benecios obtenidos del muerto, ya sea gracias a su muerte o en vida

Beneficios = Node(ConditionalProbabilityTable([
    ["Madre","Sostenimiento",0.2],
    ["Madre","Herencia",0.25],
    ["Madre","Sostenimiento y Herencia",0.15],
    ["Madre","Nada",0.4],

    ["Padre","Sostenimiento",0.2],
    ["Padre","Herencia",0.25],
    ["Padre","Sostenimiento y Herencia", 0.15],
    ["Padre","Nada", 0.4],

    ["Hijo","Sostenimiento",0.3],
    ["Hijo","Herencia",0.1],
    ["Hijo","Sostenimiento y Herencia",0.45],
    ["Hijo","Nada",0.15],

    ["Hermano","Sostenimiento",0.2],
    ["Hermano","Herencia",0.1],
    ["Hermano","Sostenimiento y Herencia",0.05],
    ["Hermano","Nada",0.65],

    ["Amigo","Sostenimiento",0.08],
    ["Amigo","Herencia",0.03],
    ["Amigo","Sostenimiento y Herencia",0.02],
    ["Amigo","Nada",0.87],

    ["Abuelo","Sostenimiento",0.14],
    ["Abuelo","Herencia",0.06],
    ["Abuelo","Sostenimiento y Herencia",0.04],
    ["Abuelo","Nada",0.76],

    ["Nieto","Sostenimiento",0.05],
    ["Nieto","Herencia",0.3],
    ["Nieto","Sostenimiento y Herencia",0.15],
    ["Nieto","Nada",0.5],

    ["Pareja","Sostenimiento",0.2],
    ["Pareja","Herencia",0.3],
    ["Pareja","Sostenimiento y Herencia",0.4],
    ["Pareja","Nada",0.1],

    ["Ninguna","Sostenimiento",0.02],
    ["Ninguna","Herencia",0.015],
    ["Ninguna","Sostenimiento y Herencia",0.005],
    ["Ninguna","Nada",0.96]

],[Relacion.distribution]), name="Beneficios")

#Emociones hacia el muerto

Emociones = Node(ConditionalProbabilityTable([
    ["Madre","Odio",0.1],
    ["Madre","Amor",0.75],
    ["Madre","Celos",0.01],
    ["Madre","Ninguno",0.14],

    ["Padre","Odio",0.1],
    ["Padre","Amor",0.75],
    ["Padre","Celos", 0.01],
    ["Padre","Ninguno", 0.14],

    ["Hijo","Odio",0.25],
    ["Hijo","Amor",0.6],
    ["Hijo","Celos",0.01],
    ["Hijo","Ninguno",0.14],

    ["Hermano","Odio",0.15],
    ["Hermano","Amor",0.5],
    ["Hermano","Celos",0.2],
    ["Hermano","Ninguno",0.15],

    ["Amigo","Odio",0.15],
    ["Amigo","Amor",0.35],
    ["Amigo","Celos",0.25],
    ["Amigo","Ninguno",0.25],

    ["Abuelo","Odio",0.1],
    ["Abuelo","Amor",0.55],
    ["Abuelo","Celos",0.03],
    ["Abuelo","Ninguno",0.32],

    ["Nieto","Odio",0.25],
    ["Nieto","Amor",0.6],
    ["Nieto","Celos",0.01],
    ["Nieto","Ninguno",0.14],

    ["Pareja","Odio",0.03],
    ["Pareja","Amor",0.65],
    ["Pareja","Celos",0.3],
    ["Pareja","Ninguno",0.02],

    ["Ninguna","Odio",0.1],
    ["Ninguna","Amor",0.1],
    ["Ninguna","Celos",0.2],
    ["Ninguna","Ninguno",0.6]

],[Relacion.distribution]), name="Emociones")

# Tiene motivos para matarlo?

Motivo = Node(ConditionalProbabilityTable([
    ["Ratero"  ,"Odio"  ,  "Sostenimiento", "Si",0.5175],
    ["Ratero"  ,"Odio"  ,  "Sostenimiento", "No",0.4825],
    ["Ratero"  ,"Odio"  ,  "Herencia", "Si",0.83375],
    ["Ratero"  ,"Odio"  ,  "Herencia", "No",0.16625],
    ["Ratero"  ,"Odio"  ,  "Sostenimiento y Herencia", "Si",0.71875],
    ["Ratero"  ,"Odio"  ,  "Sostenimiento y Herencia", "No",0.28125],
    ["Ratero"  ,"Odio"  ,  "Nada", "Si",0.618125],
    ["Ratero"  ,"Odio"  ,  "Nada", "No",0.381875],
    ["Asesino" ,"Odio"  ,  "Sostenimiento", "Si",0.585],
    ["Asesino" ,"Odio"  ,  "Sostenimiento", "No",0.415],
    ["Asesino" ,"Odio"  ,  "Herencia", "Si",0.9425],
    ["Asesino" ,"Odio"  ,  "Herencia", "No",0.0575],
    ["Asesino" ,"Odio"  ,  "Sostenimiento y Herencia", "Si",0.8125],
    ["Asesino" ,"Odio"  ,  "Sostenimiento y Herencia", "No",0.1875],
    ["Asesino" ,"Odio"  ,  "Nada", "Si",0.69875],
    ["Asesino" ,"Odio"  ,  "Nada", "No",0.30125],
    ["Neutro"  ,"Odio"  ,  "Sostenimiento", "Si",0.4275],
    ["Neutro"  ,"Odio"  ,  "Sostenimiento", "No",0.5725],
    ["Neutro"  ,"Odio"  ,  "Herencia", "Si",0.68875],
    ["Neutro"  ,"Odio"  ,  "Herencia", "No",0.31125],
    ["Neutro"  ,"Odio"  ,  "Sostenimiento y Herencia", "Si",0.59375],
    ["Neutro"  ,"Odio"  ,  "Sostenimiento y Herencia", "No",0.40625],
    ["Neutro"  ,"Odio"  ,  "Nada", "Si",0.510625],
    ["Neutro"  ,"Odio"  ,  "Nada", "No",0.489375],

    ["Ratero"  ,"Amor"  ,  "Sostenimiento", "Si",0.0828],
    ["Ratero"  ,"Amor"  ,  "Sostenimiento", "No",0.9172],
    ["Ratero"  ,"Amor"  ,  "Herencia", "Si",0.1334],
    ["Ratero"  ,"Amor"  ,  "Herencia", "No",0.8666],
    ["Ratero"  ,"Amor"  ,  "Sostenimiento y Herencia", "Si",0.115],
    ["Ratero"  ,"Amor"  ,  "Sostenimiento y Herencia", "No",0.885],
    ["Ratero"  ,"Amor"  ,  "Nada", "Si",0.0989],
    ["Ratero"  ,"Amor"  ,  "Nada", "No",0.9011],
    ["Asesino" ,"Amor"  ,  "Sostenimiento", "Si",0.0936],
    ["Asesino" ,"Amor"  ,  "Sostenimiento","No",0.9064],
    ["Asesino" ,"Amor"  ,  "Herencia", "Si",0.1508],
    ["Asesino" ,"Amor"  ,  "Herencia", "No",0.8492],
    ["Asesino" ,"Amor"  ,  "Sostenimiento y Herencia", "Si",0.13],
    ["Asesino" ,"Amor"  ,  "Sostenimiento y Herencia", "No",0.87],
    ["Asesino" ,"Amor"  ,  "Nada", "Si",0.1118],
    ["Asesino" ,"Amor"  ,  "Nada", "No",0.8882],
    ["Neutro"  ,"Amor"  ,  "Sostenimiento", "Si",0.0684],
    ["Neutro"  ,"Amor"  ,  "Sostenimiento", "No",0.9316],
    ["Neutro"  ,"Amor"  ,  "Herencia", "Si",0.1102],
    ["Neutro"  ,"Amor"  ,  "Herencia", "No",0.8898],
    ["Neutro"  ,"Amor"  ,  "Sostenimiento y Herencia", "Si",0.095],
    ["Neutro"  ,"Amor"  ,  "Sostenimiento y Herencia", "No",0.905],
    ["Neutro"  ,"Amor"  ,  "Nada", "Si",0.0817],
    ["Neutro"  ,"Amor"  ,  "Nada", "No",0.9183],

    ["Ratero"  ,"Celos" ,	 "Sostenimiento", "Si",0.5382],
    ["Ratero"  ,"Celos" ,	 "Sostenimiento", "No",0.4618],
    ["Ratero"  ,"Celos" ,	 "Herencia", "Si",0.8671],
    ["Ratero"  ,"Celos" ,	 "Herencia", "No",0.1329],
    ["Ratero"  ,"Celos" ,	 "Sostenimiento y Herencia", "Si",0.7475],
    ["Ratero"  ,"Celos" ,	 "Sostenimiento y Herencia", "No",0.2525],
    ["Ratero"  ,"Celos" ,	 "Nada", "Si",0.64285],
    ["Ratero"  ,"Celos" ,	 "Nada", "No",0.35715],
    ["Asesino" ,"Celos" ,  "Sostenimiento", "Si",0.6084],
    ["Asesino" ,"Celos" ,  "Sostenimiento", "No",0.3916],
    ["Asesino" ,"Celos" , "Herencia", "Si",0.9802],
    ["Asesino" ,"Celos" , "Herencia", "No",0.0198],  
    ["Asesino" ,"Celos" ,  "Sostenimiento y Herencia", "Si",0.845],
    ["Asesino" ,"Celos" ,  "Sostenimiento y Herencia", "No",0.155],
    ["Asesino" ,"Celos" ,  "Nada", "Si",0.7267],
    ["Asesino" ,"Celos" ,  "Nada", "No",0.2733],
    ["Neutro"  ,"Celos" , "Sostenimiento", "Si",0.4446],
    ["Neutro"  ,"Celos" , "Sostenimiento", "No",0.5554],
    ["Neutro"  ,"Celos" ,  "Herencia", "Si",0.7163],
    ["Neutro"  ,"Celos" ,  "Herencia", "No",0.2837],
    ["Neutro"  ,"Celos" ,  "Sostenimiento y Herencia", "Si",0.6175],
    ["Neutro"  ,"Celos" ,  "Sostenimiento y Herencia", "No",0.3825],
    ["Neutro"  ,"Celos" ,  "Nada", "Si",0.53105],
    ["Neutro"  ,"Celos" ,  "Nada", "No",0.46895],

    ["Ratero"  ,"Ninguno", "Sostenimiento", "Si",0.284625],
    ["Ratero"  ,"Ninguno", "Sostenimiento", "No",0.715375],
    ["Ratero"  ,"Ninguno", "Herencia", "Si",0.4585625],
    ["Ratero"  ,"Ninguno", "Herencia", "No",0.5414375],
    ["Ratero"  ,"Ninguno", "Sostenimiento y Herencia", "Si",0.3953125],
    ["Ratero"  ,"Ninguno", "Sostenimiento y Herencia", "No",0.6046875],
    ["Ratero"  ,"Ninguno", "Nada", "Si",0.33996875],
    ["Ratero"  ,"Ninguno", "Nada", "No",0.66003125],
    ["Asesino" ,"Ninguno", "Sostenimiento", "Si",0.32175],
    ["Asesino" ,"Ninguno", "Sostenimiento", "No",0.67825],
    ["Asesino" ,"Ninguno", "Herencia", "Si",0.518375],
    ["Asesino" ,"Ninguno", "Herencia", "No",0.481625],
    ["Asesino" ,"Ninguno", "Sostenimiento y Herencia", "Si",0.446875],
    ["Asesino" ,"Ninguno", "Sostenimiento y Herencia", "No",0.553125],
    ["Asesino" ,"Ninguno", "Nada", "Si",0.3843125],
    ["Asesino" ,"Ninguno", "Nada", "No",0.6156875],
    ["Neutro"  ,"Ninguno", "Sostenimiento", "Si",0.235125],
    ["Neutro"  ,"Ninguno", "Sostenimiento", "No",0.764875],
    ["Neutro"  ,"Ninguno", "Herencia", "Si",0.3788125],
    ["Neutro"  ,"Ninguno", "Herencia", "No",0.6211875],
    ["Neutro"  ,"Ninguno", "Sostenimiento y Herencia", "Si",0.3265625],
    ["Neutro"  ,"Ninguno", "Sostenimiento y Herencia", "No",0.6734375],
    ["Neutro"  ,"Ninguno", "Nada", "Si",0.28084375],
    ["Neutro"  ,"Ninguno", "Nada", "No",0.71915625]


],[Antecedentes.distribution, Emociones.distribution, Beneficios.distribution]), name="Motivo")

Asesino = Node(ConditionalProbabilityTable([

    ["Si" ,	"Alcohol", "Culpable", 0.9],
    ["Si" ,	"Alcohol", "Inocente",0.1],
    ["Si" ,	"Droga", "Culpable", 0.95],
    ["Si" ,	"Droga", "Inocente", 0.05],
    ["Si" ,	"Normal", "Culpable", 0.8],
    ["Si" ,	"Normal", "Inocente",0.2],
    ["No" ,	"Alcohol", "Culpable", 0.2],
    ["No" ,	"Alcohol", "Inocente",0.8],
    ["No" ,	"Droga", "Culpable", 0.35],
    ["No" ,	"Droga", "Inocente", 0.65],
    ["No"	, "Normal", "Culpable", 0.05],
    ["No"	, "Normal", "Inocente", 0.95]

],[Motivo.distribution, Consumo.distribution]), name="Asesino")


# Creamos una Red Bayesiana y añadimos estados
modelo = BayesianNetwork()
#modelo.add_states(Antecedentes, Relacion, Consumo, Beneficios, Emociones, Motivo, Asesino)
modelo.add_states(Antecedentes, Relacion, Consumo, Beneficios,Emociones, Motivo, Asesino)

# Añadimos bordes que conecten nodos
modelo.add_edge(Relacion, Emociones)
modelo.add_edge(Relacion, Beneficios)
modelo.add_edge(Antecedentes, Motivo)
modelo.add_edge(Emociones, Motivo)
modelo.add_edge(Beneficios, Motivo)
modelo.add_edge(Motivo, Asesino)
modelo.add_edge(Consumo, Asesino)

#Modelo Final
modelo.bake()

In [ ]:
# Calculemos la probabilidad para una observacion dada
probabilidad1 = modelo.probability([["Neutro", "Ninguna", "Normal", "Nada", "Ninguno", "No", "Inocente"]])
#probabilidad2 = modelo.probability([["fuerte", "si", "a tiempo", "atendida"]])
#probabilidad3 = modelo.probability([["suave", "no", "a tiempo", "atendida"]])

print("Probabilidad #1:",probabilidad1)
#print("Probabilidad #2:",probabilidad2)
#print("Probabilidad #3:",probabilidad3)

In [ ]:
# Calculemos las predicciones
info_podrick = ['Hijo', 'Odio', 'Sostenimiento']
info_teresa = ['Amigo', 'Celos', 'Nada']
info_alfred = ['Neutro', 'Ninguna', 'Nada']
info_elon = ['Amigo', 'Celos', 'Herencia']
info_jeff = ['Hermano', 'Odio', 'Sostenimiento y Herencia']
info_greta = ['Pareja', 'Amor', 'Sostenimiento y Herencia']


presuntos = [
   ["Podrick",info_podrick], 
   ["Teresa",info_teresa], 
   ["Alfred",info_alfred],
   ["Elon",info_elon],
   ["Jeff",info_jeff],
   ["Greta",info_greta]
]

# Para que solo calcule la probabilidad de los últimos 3 sospechosos
calcular_prob = []
for i in range(6):
    for j in range(len(sospechosos)):
       if str(sospechosos[j]) == presuntos[i][0]:
            calcular_prob.append(presuntos[i])


for sospechoso in calcular_prob:
  print()
  print("Probabilidad para: ",sospechoso[0])

  predicciones = modelo.predict_proba({
    'Relacion' : sospechoso[1][0],
    'Emociones': sospechoso[1][1],
    'Beneficios': sospechoso[1][2]
    })
  # Visualizemos las predicciones para cada nodo
  for nodo, prediccion in zip(modelo.states, predicciones):
      if isinstance(prediccion, str):
          print(f"{nodo.name}: {prediccion}")
      else:
          print(f"{nodo.name}")
          for valor, probabilidad in prediccion.parameters[0].items(): 
            print(f"       {valor}: {probabilidad:.2f}")


